In [1]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests as rq
import json
import time

# Login to ResearchGate

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

driver = webdriver.Firefox()
driver.get("https://www.researchgate.net/login")

password = driver.find_element(By.ID, "input-password")
username = driver.find_element(By.ID, "input-login")

password.send_keys("kcaeDHT8uE5wYMe")
username.send_keys("matanki@my.hit.ac.il")
password.send_keys(Keys.RETURN)

In [10]:
fullname = []
degree = []
nation = []
institute = []
depratment = []
disciplines = []
publications = []
citations = []
recommendations = []
top_cited_research = []
top_cited_research_journal = []
h_index = []

## List of institues

In [11]:
# driver.get("https://www.researchgate.net/institution/Tel_Aviv_University/members")
driver.get("https://www.researchgate.net/institution/Holon-Institute-of-Technology/members")

### Number of member pages for each institue

In [32]:
content = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/div[1]/div/div/div[2]/div/div[1]/div/footer/div/div/nav/div[7]')
index = bs(content.get_property("innerHTML"), "html.parser").get_text()
print(index)

63


In [34]:
for i in range(2, int(index)):
    driver.get("https://www.researchgate.net/institution/Holon-Institute-of-Technology/members/"+str(i))
    for j in range(1, 11):
        
        name = driver.find_element(By.XPATH, "/html/body/div[1]/div[3]/div[1]/div/div/div[2]/div/div[1]/div/div[2]/div/div["+str(j)+"]/div/div/div[2]/div/div[1]/div/div/h5/a")
        name = bs(name.get_property("innerHTML"), "html.parser").get_text()
        print(name)
        
        try: 
            department = driver.find_element(By.XPATH, "/html/body/div[1]/div[3]/div[1]/div/div/div[2]/div/div[1]/div/div[2]/div/div["+str(j)+"]/div/div/div[2]/div/div[2]/div/div[2]/ul/li/span")
            department = bs(department.get_property("innerHTML"), "html.parser").get_text()
            print(department)
        except:
            print("Nan")
        
        try: 
            disciplines = driver.find_element(By.XPATH, "/html/body/div[1]/div[3]/div[1]/div/div/div[2]/div/div[1]/div/div[2]/div/div["+str(j)+"]/div/div/div[2]/div/div[3]/div/div[2]")
            disciplines = bs(disciplines.get_property("innerHTML"), "html.parser").get_text()
            print(disciplines)
        except:
            print("Nan")
            
        print("\n")
        
        
    time.sleep(2)

Mark Korenblit
Department of Computer Science
Graph ΤheoryGraphsCombinatorial Optimization


Arik Sadeh
Technology Management Faculty
Project ManagementOrganizational LearningManagement


Amnon Fruchtman
Faculty of Sciences
SpectroscopySpace SciencesEnergy


Amir Handelman
Faculty of Engineering
Digital Signal ProcessingSignal ProcessingElectronics  and Communication Engineering


Iaakov Exman
Faculty of Sciences
Software EngineeringQuantum Software TheoryAlgebraic Software Theory


M. Elena Luna-Elizarrarás
Department of Applied Mathematics
AnalysisFunctional AnalysisAlgebra


Elad Segev
Department of Applied Mathematics
Molecular Dynamics SimulationMolecular ModelingComputational Chemistry


Yulia Kempner
Department of Computer Science
Discrete MathematicsCombinatoricsAlgorithm Development


Hagit Meishar-Tal
Instructional Technologies faculty
E-LearningTeaching and LearningPedagogy and Education


Igor Lapsker
Department of Sciences
Thin Films and NanotechnologyThin Film DepositionM

A. Ioinovici
Nan
Nan


Bruria Haberman
Department of Computer Science
PedagogyTeaching and LearningPedagogy and Education


Motti Haridim
Faculty of Engineering
AntennasAntennas and PropagationElectromagnetics


Orna Bar-Ness
Faculty of Management of Technology
Online LearningInstructional TechnologyConflict Negotiation


Samary Baranov
Department of Computer Science
Digital System DesignHigh Level SynthesisAutomata


Ben Hababo
Department of Computer Science
Nan


Shai Rozenes
Faculty of Management of Technology
Operations ManagementServiceService Management


Uri Mahlab
Faculty of Engineering
Digital Signal ProcessingSignal ProcessingSignal Analysis


Sayfan giulia Borghini
Faculty of Design
Design ThinkingDesign TheoryCreativity and Innovation


Yael Eylat Van-Essen
Nan
Nan


Sigal koral kordova
Faculty of Management of Technology
ProductionOperations ManagementOptimization


R. E. Goot
Department of Applied Mathematics
Electrical EngineeringCommunication EngineeringChannels


Shay 

Rotem Halimi
Department of Computer Science
Nan


Polina Shcherbakova
Faculty of Engineering
Electrical Power Engineering


Mor Wainberg
Department of Computer Science
FMF


Noga Gulst
school of Multi discipliner studies
Nan


Gaya Grinberg
Department of Computer Science
Solar Activity


Zehavit Kehat
Department of Computer Science
Nan


Danielle Gerty
Department of Applied Mathematics
FlowNumerical Simulation


Orel Burak
Electrical engineering
Electrical & Electronics Engineering


Rotem Gershon
Department of Computer Science
Kidney


Omri Mizrahi
Faculty of Engineering
UltrasoundDoppler UltrasonographyImaging


Netzah Calamaro
power electronics
MATLAB Simulation


Ofir Inbar
Instructional Technologies Dept
Nan


Ploni Almoni
Instructional Technologies Dept
Nan


Avi Dantess
Instructional Technologies Dept
Nan


A. Zak
Nan
Nan


A. Ben-David
Nan
Nan


Ira Mely
Department of Computer Science
Nan


Shlomi Eliyahu
Department of Applied Mathematics
Electronics  and Communication Engineer

KeyboardInterrupt: 

In [7]:
driver.get("https://www.researchgate.net/profile/Susan-Holmes-4/stats")

In [13]:
fullname

[]

בעיה: אותם נתיבים class ים למאפיינים שונים

In [22]:
content = driver.find_element(By.XPATH, '//div[@class="nova-legacy-e-text nova-legacy-e-text--size-xl nova-legacy-e-text--family-display nova-legacy-e-text--spacing-none nova-legacy-e-text--color-grey-900"]').text
fullname += [content]

content = driver.find_element(By.XPATH, '//div[@class="nova-legacy-e-link nova-legacy-e-link--color-inherit nova-legacy-e-link--theme-silent"]').text
degree += [content]

content = driver.find_element(By.XPATH, '//div[@class="nova-legacy-e-text nova-legacy-e-text--size-m nova-legacy-e-text--family-sans-serif nova-legacy-e-text--spacing-none nova-legacy-e-text--color-grey-700"]').text
nation += [content]

content = driver.find_element(By.XPATH, '//a[@class="nova-legacy-e-link nova-legacy-e-link--color-inherit nova-legacy-e-link--theme-silent"]').text
institute += [content]

content = driver.find_element(By.XPATH, '//a[@class="nova-legacy-e-link nova-legacy-e-link--color-inherit nova-legacy-e-link--theme-silent"]').text
department += [content]

content = driver.find_element(By.XPATH, '//a[@class="nova-legacy-e-link nova-legacy-e-link--color-inherit nova-legacy-e-link--theme-silent"]').text
disciplines += [content]

content = driver.find_element(By.XPATH, '//div[@class="nova-legacy-e-text nova-legacy-e-text--size-m nova-legacy-e-text--family-sans-serif nova-legacy-e-text--spacing-none nova-legacy-e-text--color-inherit"]').text
citations += [content]

content = driver.find_element(By.XPATH, '//div[@class="nova-legacy-e-text nova-legacy-e-text--size-xxxl nova-legacy-e-text--family-display nova-legacy-e-text--spacing-none nova-legacy-e-text--color-inherit"]').text
recommendations += [content]

content = driver.find_element(By.XPATH, '//div[@class="nova-legacy-e-text nova-legacy-e-text--size-m nova-legacy-e-text--family-sans-serif nova-legacy-e-text--spacing-none nova-legacy-e-text--color-inherit"]').text
h_index += [content]

content = driver.find_element(By.XPATH, '//a[@class="nova-legacy-e-link nova-legacy-e-link--color-inherit nova-legacy-e-link--theme-silent"]').text
top_cited_research += [content]

content = driver.find_element(By.XPATH, '//a[@class="nova-legacy-e-link nova-legacy-e-link--color-inherit nova-legacy-e-link--theme-silent"]').text
top_cited_research_journal += [content]

In [13]:
df = pd.DataFrame({"שם הספר":fullname, "מחבר":author, "עלות (בתוספת מע\"מ)":price})
df

,שם הספר,מחבר,"עלות (בתוספת מע""מ)"
0,נשף המשוגעות,ויקטוריה מס,79.2 ₪
1,מיומנה של אישה מתה,גואל פינטו,88.2 ₪
2,אהובת הצייר,דר פלוכט סימונה ואן,84.6 ₪
3,שיעורים בכימיה,בוני גרמוס,93.6 ₪
4,והגיענו לזמן הזה,עידן סגר,88.2 ₪
5,על השעות השבורות,ריקי כהן,68.4 ₪
6,צרוב בגוף,ישראל המאירי,75.6 ₪
7,אילו נולד איטלקי,שי אספריל,84.6 ₪
8,אדונית השקרנים,אן פאצט,88.2 ₪
9,העמק הירוק האחרון,מארק סאליבן,88.2 ₪


In [14]:
df.to_csv('book_prices')